In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import uproot
import matplotlib as mpl
import math

from scipy.optimize import curve_fit
from scipy import interpolate, special
from scipy.stats import norm as normal

import cProfile

import inspect

import os

In [ ]:
RUNS = [9133]
TPCs = ["EE","EW","WE","WW"]
IPLANE = 2

THETA = 50
THETA_HI = THETA + 2
THETA_LOS = np.array(range(60, 80, 2))
THETA_HIS = THETA_LOS + 2

In [ ]:
data_files = ["./data/baseline-fix/%i/WFresults_Plane%i_%s.root" % (RUN, IPLANE, TPC) for RUN in RUNS for TPC in TPCs]

# data_files = [ "./mc/far_TrackBased_official/WFresults_Plane%i_XX.root" % IPLANE]

data_hfiles = [uproot.open(f) for f in data_files]
data_hs = [hf["AnodeRecoHist1D_%ito%i" % (tlo, thi)] for hf in data_hfiles for (tlo, thi) in zip (THETA_LOS, THETA_HIS)]
data_h_val = np.average([h.values() for h in data_hs], axis=0)
data_h_center = data_hs[0].axis().centers()

In [ ]:
def norm(v, vnorm=None):
    if vnorm is None:
        vnorm = v
    if IPLANE == 2:
        return v / vnorm.max()
    else:
        return v / np.abs(vnorm.min())

In [ ]:
baseline = np.average(data_h_val[:50])
data_h_val -= baseline

baseline

In [ ]:
plt.plot(data_h_center, norm(data_h_val), linestyle="none", marker=".", label="Data")
#plt.xlim([-10, 10])
plt.legend()
plt.axhline([0], color="r")

In [ ]:
def exp(t, a, t0):
    return a*np.exp(-t/t0)*(t > 0)

In [ ]:
p0 = [0.1, 65]

popt, perr = curve_fit(exp, data_h_center[data_h_center>40], norm(data_h_val)[data_h_center>40], p0=p0)
popt[1]

In [ ]:
tau = popt[1]

t2=80
t1=40

area_tail = np.sum(data_h_val[data_h_center>t1])

frac_area = area_tail / np.sum(data_h_val)

RCA = area_tail*np.exp((t1+t2)/tau) / (np.exp(t2/tau) - np.exp(t1/tau))

frac_area_to_inf = RCA / (np.sum(data_h_val) + RCA*np.exp(-t2/tau))

frac_area, frac_area_to_inf

In [ ]:
plt.plot(data_h_center, norm(data_h_val), linestyle="none", marker=".", label="Data")
#plt.xlim([-10, 10])
plt.plot(data_h_center[data_h_center>0], exp(data_h_center[data_h_center>0], *popt), label="RC Tail Fit")

plt.legend()
plt.yscale("log")
plt.text(0.6, 0.125, "RC: %.1f$\\mu$s\n$\\frac{\\sum wvf(t > %.0f\\mu s)}{\\sum wvf}$: %.3f\n$\\frac{\\int_0^\\infty A exp[-t/RC]}{\\sum wvf + \\int_{%.0f}^\\infty A exp[-t/RC]}$: %.3f" % (popt[1], t1, frac_area, t2, frac_area_to_inf), fontsize=12, transform=plt.gca().transAxes)
plt.title("Averaged Waveform Data")
plt.ylabel("Normalized Amplitude")
plt.xlabel("Time [$\\mu$s]")

In [ ]:
plt.plot(data_h_center, (data_h_val) / data_h_val.sum()/(1-0.161), linestyle="none", marker=".", label="Data")
#plt.xlim([-10, 10])
plt.plot(data_h_center[data_h_center>0], exp(data_h_center[data_h_center>0], *popt), label="RC Tail Fit")

plt.legend()
plt.yscale("log")
plt.text(0.6, 0.125, "RC: %.1f$\\mu$s\n$\\frac{\\sum wvf(t > %.0f\\mu s)}{\\sum wvf}$: %.3f\n$\\frac{\\int_0^\\infty A exp[-t/RC]}{\\sum wvf + \\int_{%.0f}^\\infty A exp[-t/RC]}$: %.3f" % (popt[1], t1, frac_area, t2, frac_area_to_inf), fontsize=12, transform=plt.gca().transAxes)
plt.title("Averaged Waveform Data")
plt.ylabel("Normalized Amplitude")
plt.xlabel("Time [$\\mu$s]")